In [24]:
from src.DataLoader import gdfLoader,ReceiverLoader,RSSIDataLoader
from func.utils.getMarkerDictList import getMarkerDictList
from src.DataLoader import Room,Receiver

In [25]:
#Map Data
MapDataPath="data/cartePalaiseau.csv"
map_gdf=gdfLoader(MapDataPath)
#Receiver Data
ReceiverDataPath="data/carteModulePalaiseau.csv"
receiver_gdf=ReceiverLoader(ReceiverDataPath)

In [26]:
receiver_gdf

,macModule,geometry,coordinatesEPSG3857
0,A8032A31204E,POINT (2.20079 48.71316),POINT (244990.444 6226329.680)
1,C45BBE39F42A,POINT (2.20110 48.71315),POINT (245025.333 6226329.187)
2,C45BBE39F9AE,POINT (2.20120 48.71314),POINT (245035.947 6226326.828)
3,A8032A311FAA,POINT (2.20072 48.71317),POINT (244983.036 6226331.224)
4,C45BBE39F56A,POINT (2.20117 48.71318),POINT (245033.645 6226333.296)
5,C45BBE37B346,POINT (2.20087 48.71315),POINT (244999.412 6226328.251)
6,C45BBE386952,POINT (2.20098 48.71316),POINT (245011.849 6226330.207)
7,C45BBE386362,POINT (2.20094 48.71314),POINT (245007.682 6226326.760)
8,C45BBE384CD2,POINT (2.20093 48.71317),POINT (245006.234 6226331.144)
9,C45BBE39870E,POINT (2.20106 48.71315),POINT (245020.561 6226328.967)


In [27]:
from src.DataLoader import room_dict_generator,room_r_tree_generator
room_dict=room_dict_generator(map_gdf)
room_r_tree=room_r_tree_generator(room_dict)


In [28]:
from src.DataLoader import build_receiver_dict
receiver_dict= build_receiver_dict(receiver_gdf,room_dict,room_r_tree)


In [29]:
Threshold1=-45
Threshold2=-70

In [62]:
from src.DataLoader import Receiver
from shapely.geometry import Point,Polygon
from src.utils.geometry import discrete_circle,intersection
import rtree

#need in the class: receiver_dict
def zone_selection(receiver:Receiver,rssi:int,room_r_tree:rtree,room_dict:dict)->Polygon:
    """
    Return the predicted zone for the selected receiver receiving the rssi 
    
    Parameters
    ---
    Input:
    receiver: Receiver
    rssi: int
    
    Output:
    Polygon
    """
    if rssi>Threshold1:
        #return the room in wich the receiver is
        return [receiver.room.cartesian_polygon]
    
    elif Threshold2< rssi < Threshold1:
        #return all the room in wich the receiver is
        point=receiver.cartesian_point
        area=discrete_circle(6,[point.x,point.y],5)#create an hexagon with 6 points centered on the receiver
        rooms=list(intersection(area,room_r_tree,room_dict))
        return rooms
    elif rssi<Threshold2:
        point=receiver.cartesian_point
        area=discrete_circle(6,[point.x,point.y],10)#create an hexagon with 6 points centered on the receiver
        rooms=list(intersection(area,room_r_tree,room_dict))
        return rooms    

In [64]:
from shapely import equals_exact
def test_case1(receiver_dict):
    receiver=receiver_dict['A8032A31204A']
    rssi=-40
    zone=zone_selection(receiver,rssi,room_r_tree,room_dict)
    assert len(zone)==1,f'Error: point in more than one room'
    zone=zone[0]
    assert equals_exact(zone,receiver.room.cartesian_polygon),f'Error: {zone} should be {receiver.room.cartesian_polygon}'
test_case1(receiver_dict)
def test_full_treshold(receiver_dict):
    for receiver in receiver_dict.values():
        rssi=-40
        zone=zone_selection(receiver,rssi,room_r_tree,room_dict)
        assert len(zone)==1,f'Error: point in more than one room'
        zone=zone[0]
        assert equals_exact(zone,receiver.room.cartesian_polygon),f'Error: {zone} should be {receiver.room.cartesian_polygon}'
test_full_treshold(receiver_dict)

In [65]:
from src.visualization import MapCreation

map_object=MapCreation(receiver_dict,room_dict)


In [66]:
map_object

In [67]:
import ast
import os
from src.DataLoader import gdfLoader,ReceiverLoader
from src import API
from src.preprocessing import Preprocessor
from src.preprocessing import remove_duplicates_Cleaner,moving_average_Filter,moving_max_Filter,moving_max_averaged_Filter,blank_Filter
import pandas as pd
from src.visualization import rssi_viewer

#API call
callApi=API()
start="2023-07-27 12:04:00.000000"
end="2023-07-27 12:06:00.000000"
mac_module=ast.literal_eval(os.environ['MAC_WEAR_DICT'])['3480']


raw_data=callApi.getRawDataForCartoWear(mac_module,pd.to_datetime(start),pd.to_datetime(end))
#Preprocessing
sampling_time=1
window='10S'
preprocessor=Preprocessor(raw_data,sampling_time=sampling_time)
rssi_df=preprocessor.set_cleaner(remove_duplicates_Cleaner())\
        .set_filter(moving_max_Filter("2s"))\
        .process()
fig=rssi_viewer(rssi_df)


INFO - data Cleaning


In [68]:
rssi_df.dropna(inplace=True)
rssi_df

,timestamp,macModule,rssi
1,2023-07-27 10:04:01.500,A8032A311F96,-79.0
2,2023-07-27 10:04:01.500,A8032A311DAA,-75.0
3,2023-07-27 10:04:02.500,A8032A311DAA,-75.0
4,2023-07-27 10:04:02.500,A8032A311F96,-79.0
5,2023-07-27 10:04:03.000,A8032A311F6A,-72.0
...,...,...,...
426,2023-07-27 10:05:53.500,A8032A311F96,-79.0
428,2023-07-27 10:05:54.500,A8032A311F96,-79.0
430,2023-07-27 10:05:55.500,A8032A311F96,-79.0
431,2023-07-27 10:05:56.000,A8032A311F6A,-57.0


In [69]:
fig=rssi_viewer(rssi_df)


In [70]:
for row in rssi_df.iterrows():
    print(zone_selection(receiver_dict[row[1]['macModule']],row[1]['rssi'],room_r_tree,room_dict)) 

StopIteration: 